# Produce train from OSM2LULC results

In [ ]:
fishnets = '/mnt/d/gwork/osm2lulc_imgcls/lisboa_fishnet/'
intersections = '/mnt/d/gwork/osm2lulc_imgcls/r_lisboa_results/'
results = '/mnt/d/gwork/osm2lulc_imgcls/lisboa_results/'

In [ ]:
import os
import pandas as pd
from glass.pyt.oss import lst_ff, fprop
from glass.gt.fmshp import shp_to_obj
from glass.gt.toshp import df_to_shp

In [ ]:
fish_files = lst_ff(fishnets, file_format='.shp')

In [ ]:
for fishp in fish_files:
    # List Intersection files for each fishnet
    int_files = lst_ff(os.path.join(
        intersections, fprop(fishp, 'fn')
    ), file_format='.shp')
    
    if not len(int_files):
        continue
    
    # Open Fishnet
    fish_df = shp_to_obj(fishp)
    fish_df.rename(columns={'FID' : 'fid'}, inplace=True)
    fish_df['area'] = fish_df.geometry.area
    
    # Open Other files
    for f in int_files:
        fn = fprop(f, 'fn')
        
        df = shp_to_obj(f)
        
        if fn != 'ovl_union':
            df = df[~df.b_lulc.isnull()]
        else:
            df = df[~df.b_refid.isnull()]
        
        if fn == 'ovl_union':
            df['areav'] = df.geometry.area
        
        df = pd.DataFrame({'areav' : df.groupby(['a_FID'])['areav'].agg('sum')}).reset_index()
        
        fish_df = fish_df.merge(df, how='left', left_on='fid', right_on='a_FID')
        
        if fn != 'ovl_union':
            fish_df[fn] = fish_df.areav * 100 / fish_df.area
        else:
            fish_df['overlay'] = fish_df.areav * 100 / fish_df.area
        
        fish_df.drop(['areav', 'a_FID'], axis=1, inplace=True)
    
    # Save file
    df_to_shp(fish_df, os.path.join(results, os.path.basename(fishp)))

In [ ]:
rst = '/home/jasp/mrgis/landsense_pp/rst_pnse.tif'
fishnets = '/home/jasp/mrgis/landsense_pp/tmp_pnse'

from glass.gt.sample import nfishnet_fm_rst

fnet = nfishnet_fm_rst(rst, 500, 500, fishnets)